## VARS DwC conversion - larger data set (all species observed in 2001)

Note that initially, this code was run to convert data from 2001. Since then, it has been run on other years (2010, 2017, 1989) to expand list of species names incorrectly matched to WoRMS that need to be managed.

Resources:
- https://dwc.tdwg.org/terms/
- https://tools.gbif.org/dwca-validator/extension.do?id=dwc:Occurrence
- https://www.mbari.org/products/research-software/video-annotation-and-reference-system-vars/query-interface/advanced-user-guide/
- https://www.gbif.org/data-quality-requirements-occurrences

In [1]:
## Imports

import pandas as pd
import numpy as np

import re # for extracting logon info from text file

import jaydebeapi # for connecting to VARS db
import VARS # for connecting to VARS db

from datetime import datetime # for handling dates
import pytz # for handling time zones

import urllib.request, urllib.parse, json # for dealing with WoRMS API and output
import WoRMS # functions for querying WoRMS REST API

### Obtain data from VARS database

In [30]:
# ## Extract logon information from text file

# # Get list of each line in file
# filename = 'VARS_logon_info.txt'
# f = open(filename, 'r')
# lines = f.readlines()
# f.close()

# # Function for extracting information from lines
# def get_single_quoted_text(s):
#     """ 
#     Takes string s and returns any text in s that is between the first set of single quotes, removing whitespace. 
    
#     Example:
#     s = "What if there's more ' than one' sest of single' quotes?"
#     get_single_quoted_text(s) --> 's more'
    
#     """
    
#     extracted_text = re.search('''(?<=')\s*[^']+?\s*(?=')''', s)
#     return(extracted_text.group().strip())

# # Assign logon info
# dr = get_single_quoted_text(lines[2])
# name = get_single_quoted_text(lines[3])
# pw = get_single_quoted_text(lines[4])
# un = get_single_quoted_text(lines[5])
# url = get_single_quoted_text(lines[6])

An explanation of the regex in get_single_quoted_text() can be found here: <br>
https://stackoverflow.com/questions/42002931/regex-extract-string-between-single-quotes-trim-whitespace?rq=1

In [36]:
# ## Build SQL query

# sql = """
#         SELECT index_recorded_timestamp,
#                observation_uuid,
#                concept,
#                observation_group,
#                observer,
#                image_url,
#                depth_meters,
#                latitude,
#                longitude,
#                oxygen_ml_per_l,
#                psi,
#                salinity,
#                temperature_celsius,
#                video_uri,
#                video_sequence_name,
#                chief_scientist
#         FROM annotations a
#         WHERE NOT EXISTS (
#            SELECT DISTINCT observation_uuid
#            FROM annotations b
#            WHERE (
#              (  -- Delete last 2 years of annotations
#              index_recorded_timestamp > DATEADD([year], - 2, GETDATE()) OR
#              index_recorded_timestamp IS NULL OR
#              index_recorded_timestamp < CAST('1970-01-02' AS datetime)
#              )
#            OR ( -- Delete embargoes by dive
#              dive_number IN ('Ventana 50', 'Ventana 217', 'Ventana 218', 'Ventana 248')
#               )
#            OR (
#              dive_number IN ('Tiburon 1001', 'Tiburon 1029', 'Tiburon 1030', 'Tiburon 1031', 'Tiburon 1032', 'Tiburon 1033', 'Tiburon 1034')
#              )
#            OR ( -- Delete embargoes by selectedConcept
#              concept IN (
#                  'Aegina sp. 1',
#                  'Ctenophora',
#                  'Cydippida 2',
#                  'Cydippida',
#                  'Intacta',
#                  'Llyria',
#                  'Lyrocteis',
#                  'Lyroctenidae',
#                  'Mertensia',
#                  'Mertensiidae sp. A',
#                  'Mystery Mollusc',
#                  'Mystery Mollusc',
#                  'Physonectae sp. 1',
#                  'Platyctenida sp. 1',
#                  'Platyctenida',
#                  'Thalassocalycida sp. 1',
#                  'Thalassocalycida',
#                  'Thliptodon sp. A',
#                  'Tjalfiella tristoma',
#                  'Tjalfiella',
#                  'Tjalfiellidae',
#                  'Tuscarantha braueri',
#                  'Tuscarantha luciae',
#                  'Tuscarantha',
#                  'Tuscaretta globosa',
#                  'Tuscaretta',
#                  'Tuscaridium cygneum',
#                  'Tuscaridium',
#                  'Tuscarilla campanella',
#                  'Tuscarilla nationalis',
#                  'Tuscarilla similis',
#                  'Tuscarilla',
#                  'Tuscarora',
#                  'Tuscaroridae'
#                  )
#             )
#         ) AND a.observation_uuid = b.observation_uuid
#     ) AND index_recorded_timestamp >= CAST('1989-01-01' AS datetime) 
#       AND index_recorded_timestamp <= CAST('1989-12-31' AS datetime)
#     """

In [37]:
# ## Query the database

# # Get connection
# conn = VARS.get_db_conn(dr, url, un, pw, name)

# # Submit query
# data = VARS.get_data(conn, sql)

# # Close connection
# conn.close()

In [38]:
# ## Check data is there

# print(data.shape)
# data.head()

(44482, 16)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1989-12-08 18:28:49,B47AB24B-CF05-4611-9762-68D373236C7F,Nanomia bijuga,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter
1,1989-12-08 18:29:16,B13CDE3D-C6B3-43CE-A827-4C56C9A0AE1B,Nanomia bijuga,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter
2,1989-12-08 18:44:02,5465C311-816E-4C79-B87B-71C5AF30B161,Sebastes,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter
3,1989-12-08 18:40:14,2894087F-AD6C-499D-8897-3C831D7A34D8,rock,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter
4,1989-12-08 18:33:03,2D19F6CE-E673-4403-8345-CB7200332B94,Eusergestes similis,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter


**Note:** For some reason, this query didn't return any column names. I'll add them here...

In [39]:
# ## Add column names

# data.rename(columns={
#     0:'index_recorded_timestamp',
#     1:'observation_uuid',
#     2:'concept',
#     3:'observation_group',
#     4:'observer',
#     5:'image_url',
#     6:'depth_meters',
#     7:'latitude',
#     8:'longitude',
#     9:'oxygen_ml_per_l',
#     10:'psi',
#     11:'salinity',
#     12:'temperature_celsius',
#     13:'video_uri',
#     14:'video_sequence_name',
#     15:'chief_scientist'
# }, inplace=True)

# data.head()

,index_recorded_timestamp,observation_uuid,concept,observation_group,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri,video_sequence_name,chief_scientist
0,1989-12-08 18:28:49,B47AB24B-CF05-4611-9762-68D373236C7F,Nanomia bijuga,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter
1,1989-12-08 18:29:16,B13CDE3D-C6B3-43CE-A827-4C56C9A0AE1B,Nanomia bijuga,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter
2,1989-12-08 18:44:02,5465C311-816E-4C79-B87B-71C5AF30B161,Sebastes,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter
3,1989-12-08 18:40:14,2894087F-AD6C-499D-8897-3C831D7A34D8,rock,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter
4,1989-12-08 18:33:03,2D19F6CE-E673-4403-8345-CB7200332B94,Eusergestes similis,ROV,unknown,None,NaN,NaN,NaN,None,NaN,NaN,NaN,urn:tid:mbari.org:V0105-03,Ventana 0105,Chuck Baxter


In [40]:
# ## Save

# data.to_csv('VARS_1989_data.csv', index=False, na_rep='NaN')

### Read in saved data (if not pulled directly from the database)

In [2]:
## Load csv

path = ''
filename = 'VARS_2001_data.csv'
data = pd.read_csv(path+filename, dtype={'image_url': object})

print(data.shape)
data.head()

(210576, 16)


,index_recorded_timestamp,observation_uuid,concept,observation_group,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri,video_sequence_name,chief_scientist
0,2001-05-14 23:50:03,53CF02AF-535F-41AE-B31E-4BCAB4F39A56,manipulator,ROV,vars,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,1948.599976,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,urn:tid:mbari.org:T0324-09,Tiburon 0324,David Clague
1,2001-05-14 23:50:03,E9881AC2-AD21-4420-B2DD-3C4A73645E92,Holothuroidea,ROV,vars,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,1948.599976,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,urn:tid:mbari.org:T0324-09,Tiburon 0324,David Clague
2,2001-07-05 21:16:45,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,ROV,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,98.400002,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,urn:tid:mbari.org:V2016-05,Ventana 2016,Rob Sherlock
3,2001-07-05 21:16:45,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,ROV,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,98.400002,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,urn:tid:mbari.org:V2016-05,Ventana 2016,Rob Sherlock
4,2001-07-05 21:16:45,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,ROV,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,98.400002,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,urn:tid:mbari.org:V2016-05,Ventana 2016,Rob Sherlock


### Pre-processing

In [3]:
## Drop duplicate rows that arise from associations, which we don't care about here

data = data.drop_duplicates()
print(data.shape)
data.head()

(172102, 16)


,index_recorded_timestamp,observation_uuid,concept,observation_group,observer,image_url,depth_meters,latitude,longitude,oxygen_ml_per_l,psi,salinity,temperature_celsius,video_uri,video_sequence_name,chief_scientist
0,2001-05-14 23:50:03,53CF02AF-535F-41AE-B31E-4BCAB4F39A56,manipulator,ROV,vars,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,1948.599976,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,urn:tid:mbari.org:T0324-09,Tiburon 0324,David Clague
1,2001-05-14 23:50:03,E9881AC2-AD21-4420-B2DD-3C4A73645E92,Holothuroidea,ROV,vars,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,1948.599976,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,urn:tid:mbari.org:T0324-09,Tiburon 0324,David Clague
2,2001-07-05 21:16:45,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,ROV,schlin,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,98.400002,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,urn:tid:mbari.org:V2016-05,Ventana 2016,Rob Sherlock
5,2001-10-03 17:15:41,C283DC75-BB9A-4E98-9E72-BC8EABC6EED9,rock,ROV,svonthun,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,881.400024,36.760986,-121.984398,0.37,255.600006,34.513000,4.420,urn:tid:mbari.org:V2076-02,Ventana 2076,Charlie Paull
6,2001-10-03 17:15:41,EBE21573-7B5D-422B-87ED-83CB16F1D611,ledge,ROV,svonthun,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,881.400024,36.760986,-121.984398,0.37,255.600006,34.513000,4.420,urn:tid:mbari.org:V2076-02,Ventana 2076,Charlie Paull


### Convert

In [4]:
## Start with basic event data and change headings

converted = data[['index_recorded_timestamp', 'video_sequence_name', 'observation_group', 'chief_scientist']]
converted = converted.rename(columns={
    'index_recorded_timestamp':'eventDate',
    'video_sequence_name':'eventID',
    'observation_group':'samplingProtocol',
    'chief_scientist':'recordedBy'
})
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy
0,2001-05-14 23:50:03,Tiburon 0324,ROV,David Clague
1,2001-05-14 23:50:03,Tiburon 0324,ROV,David Clague
2,2001-07-05 21:16:45,Ventana 2016,ROV,Rob Sherlock
5,2001-10-03 17:15:41,Ventana 2076,ROV,Charlie Paull
6,2001-10-03 17:15:41,Ventana 2076,ROV,Charlie Paull


A small number of records have samplingProtocol = NaN. However, they have one of the ROVs indicated in the eventID, so I feel comfortable forcing all records to ROV.

In [5]:
## Ensure samplingProtocol is always ROV

converted['samplingProtocol'] = 'ROV'
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy
0,2001-05-14 23:50:03,Tiburon 0324,ROV,David Clague
1,2001-05-14 23:50:03,Tiburon 0324,ROV,David Clague
2,2001-07-05 21:16:45,Ventana 2016,ROV,Rob Sherlock
5,2001-10-03 17:15:41,Ventana 2076,ROV,Charlie Paull
6,2001-10-03 17:15:41,Ventana 2076,ROV,Charlie Paull


In [6]:
## Remove whitespace from eventID

converted['eventID'] = [event.replace(' ', '_') for event in converted['eventID']]
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy
0,2001-05-14 23:50:03,Tiburon_0324,ROV,David Clague
1,2001-05-14 23:50:03,Tiburon_0324,ROV,David Clague
2,2001-07-05 21:16:45,Ventana_2016,ROV,Rob Sherlock
5,2001-10-03 17:15:41,Ventana_2076,ROV,Charlie Paull
6,2001-10-03 17:15:41,Ventana_2076,ROV,Charlie Paull


**Note** that this code also places an underscore between 'Doc' and 'Ricketts'. It's possible that using 'DocRicketts' could be preferable.

In [7]:
## Add datasetID

converted['datasetID'] = 'VARS'

In [8]:
## Add institutionCode

converted['institutionCode'] = 'MBARI'
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode
0,2001-05-14 23:50:03,Tiburon_0324,ROV,David Clague,VARS,MBARI
1,2001-05-14 23:50:03,Tiburon_0324,ROV,David Clague,VARS,MBARI
2,2001-07-05 21:16:45,Ventana_2016,ROV,Rob Sherlock,VARS,MBARI
5,2001-10-03 17:15:41,Ventana_2076,ROV,Charlie Paull,VARS,MBARI
6,2001-10-03 17:15:41,Ventana_2076,ROV,Charlie Paull,VARS,MBARI


In [9]:
## Format eventDate

formatted = []

for dt in converted['eventDate']:
    
    # Convert string to datetime
    try:
        dt = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S.%f') # some datetimes have milliseconds
    except ValueError:
        dt = datetime.strptime(dt, '%Y-%m-%d %H:%M:%S')
        
    # Assign UTC timezone
#     utc = pytz.UTC
#     dt = dt.astimezone(utc)
    
    # Put in ISO format string
    dt = dt.isoformat()
    
    # Save in list
    formatted.append(dt + 'Z')

converted['eventDate'] = formatted
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode
0,2001-05-14T23:50:03Z,Tiburon_0324,ROV,David Clague,VARS,MBARI
1,2001-05-14T23:50:03Z,Tiburon_0324,ROV,David Clague,VARS,MBARI
2,2001-07-05T21:16:45Z,Ventana_2016,ROV,Rob Sherlock,VARS,MBARI
5,2001-10-03T17:15:41Z,Ventana_2076,ROV,Charlie Paull,VARS,MBARI
6,2001-10-03T17:15:41Z,Ventana_2076,ROV,Charlie Paull,VARS,MBARI


In [10]:
## Add in occurrence-related columns from data, renaming as needed

converted['occurrenceID'] = data['observation_uuid']
converted['scientificName'] = data['concept']
converted['occurrenceRemarks'] = data['concept']
converted['identifiedBy'] = data['observer']
converted['minimumDepthInMeters'] = round(data['depth_meters'], 1) - 2.2 # Sensor is at most 2 m shallower than camera/observed organism (?), and sensor is accurate within 20 cm
converted['maximumDepthInMeters'] = round(data['depth_meters'], 1) + 2.2
converted['verbatimDepth'] = round(data['depth_meters'], 1)
converted['decimalLatitude'] = data['latitude']
converted['decimalLongitude'] = data['longitude']
converted['dissolvedOxygenInMLPerL'] = data['oxygen_ml_per_l']
converted['pressureInPsi'] = data['psi']
converted['salinity'] = data['salinity']
converted['temperatureInCelsius'] = data['temperature_celsius']
converted['image_url'] = data['image_url']
converted['video_uri'] = data['video_uri']
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode,occurrenceID,scientificName,occurrenceRemarks,identifiedBy,...,maximumDepthInMeters,verbatimDepth,decimalLatitude,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri
0,2001-05-14T23:50:03Z,Tiburon_0324,ROV,David Clague,VARS,MBARI,53CF02AF-535F-41AE-B31E-4BCAB4F39A56,manipulator,manipulator,vars,...,1950.8,1948.6,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0324-09
1,2001-05-14T23:50:03Z,Tiburon_0324,ROV,David Clague,VARS,MBARI,E9881AC2-AD21-4420-B2DD-3C4A73645E92,Holothuroidea,Holothuroidea,vars,...,1950.8,1948.6,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0324-09
2,2001-07-05T21:16:45Z,Ventana_2016,ROV,Rob Sherlock,VARS,MBARI,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,Forskalia,schlin,...,100.6,98.4,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2016-05
5,2001-10-03T17:15:41Z,Ventana_2076,ROV,Charlie Paull,VARS,MBARI,C283DC75-BB9A-4E98-9E72-BC8EABC6EED9,rock,rock,svonthun,...,883.6,881.4,36.760986,-121.984398,0.37,255.600006,34.513000,4.420,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2076-02
6,2001-10-03T17:15:41Z,Ventana_2076,ROV,Charlie Paull,VARS,MBARI,EBE21573-7B5D-422B-87ED-83CB16F1D611,ledge,ledge,svonthun,...,883.6,881.4,36.760986,-121.984398,0.37,255.600006,34.513000,4.420,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2076-02


In [11]:
## Add coordinateUncertaintyInMeters 

converted['coordinateUncertaintyInMeters'] = 300
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode,occurrenceID,scientificName,occurrenceRemarks,identifiedBy,...,verbatimDepth,decimalLatitude,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters
0,2001-05-14T23:50:03Z,Tiburon_0324,ROV,David Clague,VARS,MBARI,53CF02AF-535F-41AE-B31E-4BCAB4F39A56,manipulator,manipulator,vars,...,1948.6,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0324-09,300
1,2001-05-14T23:50:03Z,Tiburon_0324,ROV,David Clague,VARS,MBARI,E9881AC2-AD21-4420-B2DD-3C4A73645E92,Holothuroidea,Holothuroidea,vars,...,1948.6,21.742367,-159.504933,2.07,320.799988,34.594002,2.407,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0324-09,300
2,2001-07-05T21:16:45Z,Ventana_2016,ROV,Rob Sherlock,VARS,MBARI,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,Forskalia,schlin,...,98.4,36.702446,-122.059237,2.55,329.500000,34.019001,9.307,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2016-05,300
5,2001-10-03T17:15:41Z,Ventana_2076,ROV,Charlie Paull,VARS,MBARI,C283DC75-BB9A-4E98-9E72-BC8EABC6EED9,rock,rock,svonthun,...,881.4,36.760986,-121.984398,0.37,255.600006,34.513000,4.420,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2076-02,300
6,2001-10-03T17:15:41Z,Ventana_2076,ROV,Charlie Paull,VARS,MBARI,EBE21573-7B5D-422B-87ED-83CB16F1D611,ledge,ledge,svonthun,...,881.4,36.760986,-121.984398,0.37,255.600006,34.513000,4.420,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2076-02,300


In [12]:
## Get a list of unique species names

converted['scientificName'] = [name.lower().strip() for name in converted['scientificName']]
names = converted['scientificName'].unique()

In [13]:
## Create list of concept names that should not be in species list using WoRMS logs from original runs

def get_concept_names(filename):
    """ Takes a WoRMS log .txt file filename and returns a list of concept names that did not match in WoRMS"""
    
    # Extract data from file
    f = open(filename, 'r')
    lines = f.readlines()
    f.close()

    # Initialize list to hold non-organism concept names
    concepts = []

    # Fill list
    for line in lines:
        if line.split('  ')[0] == 'Url didn\'t work, check name:':
            concepts.append(line.split('  ')[1].strip())
        else:
            concepts.append(line.split('  ')[0].split('for ')[1].split(' checking')[0])           
                
    return concepts

concepts_1989 = get_concept_names('VARS_1989_WoRMS_log.txt')
concepts_2001 = get_concept_names('VARS_2001_WoRMS_log.txt')
concepts_2010 = get_concept_names('VARS_2010_WoRMS_log.txt')
concepts_2017 = get_concept_names('VARS_2017_WoRMS_log.txt')

# Merge these lists into a single list with no duplicate terms
concepts_1989 = list(dict.fromkeys(concepts_1989))
nonorganism_concepts = concepts_1989 + list(set(concepts_2001) - set(concepts_1989))
nonorganism_concepts = nonorganism_concepts + list(set(concepts_2010) - set(nonorganism_concepts))
nonorganism_concepts = nonorganism_concepts + list(set(concepts_2017) - set(nonorganism_concepts))
nonorganism_concepts = list(dict.fromkeys(nonorganism_concepts))

# Remove known biological terms that should have matched in WoRMS but didn't
concepts_to_remove = ['larvacean',
                      'teuthoidea',
                      'phyllospadix-zostera',
                      'gastrozooid',
                      'opisthoteuthis cf. californiana',
                      'retimohnia sp. a',
                      'hydrolagus cf. trolli',
                      'beringraja rhina',
                      'synallactidae gen. et sp. indet.',
                      'tomopterid',
                      'pleuronectiformes 2',
                      'sergestid',
                      'neptunea-buccinum',
                      'pyrosomida',
                      'sebastomus complex',
                      'luciobrotula sp. a',
                      'vitreosalpa',
                      'cirrothauma sp. 1',
                      'nuculana sp. a',
                      'ceriantharia sp. 1',
                      'scotoplanes sp. a',
                      'liparidae sp. 1',
                      'actiniidae sp. 1',
                      'opisthoteuthis sp. a',
                      'coryphaenoides armatus-yaquinae complex',
                      'lycenchelys sp. 1',
                      'bathyraja cf. microtrachys',
                      'vitreosalpa gemini',
                      'coryphaenoides armatus-leptolepis-yaquinae complex',
                      'gastropoda sp. 2',
                      'euphausia 2',
                      'funiculina-halipteris complex',
                      'levinsenia sp. a',
                      'peniagone sp. a',
                      'lrj complex',
                      'doliolinetta',
                      'grimpoteuthis sp. 5',
                      'hydromedusae',
                      'funiculina-halipteris',
                      'neptunea-buccinum complex',
                      'coryphaenoides acrolepis-filifer complex',
                      'poeobius 2',
                      'medusae',
                      'lrj',
                      'graneledoninae',
                      'sicyonis sp. 1',
                      'torquaratoridae sp. 2',
                      'sebastomus',
                      'grimpoteuthis sp. 1',
                      'ceriantharia sp. 2',
                      'hormathiidae sp. 1',
                      'ceriantharia sp. 3',
                      'polychelidae 2',
                      'aeolidiidae sp. 1',
                      'farrea truncata complex',
                      'psamminidae sp. 1',
                      'hexactinellida sp. 2',
                      'benthoctopus sp. 1',
                      'brachiopod',
                      'cydippida sp. a',
                      'tromikosoma sp. 1',
                      'tetrorchis sp. a',
                      'torquaratoridaea',
                      'cladorhizidae sp. a',
                      'oloughlinius sp. 2',
                      'torquaratoridaeb sp. 1',
                      'amphipod',
                      'sebastes aleutianus-melanostictus complex',
                      'oloughlinius sp. 1',
                      'paraliparis sp. 1',
                      'torquaratoridaeb',
                      'kophobelemnon sp. 1',
                      'doliolida 2',
                      'torquaratoridaea sp. 1',
                      'ctenophora sp. a',
                      'torquaratoridaeb sp. 2',
                      'sebastes crameri-melanostomus complex',
                      'asbestopluma monticola',
                      'porifera sp. 1',
                      'psamminidae sp. 2',
                      'asteroidea sp. 1',
                      'pannychia sp. 1',
                      'antedonoidea sp. 1',
                      'benthodytes aff. typica',
                      'octacnemus sp. 1',
                      'enteropneust',
                      'porifera sp. 4',
                      'demospongiae sp. 1',
                      'candelabridae sp. 1',
                      'holothuroidea sp. 8',
                      'hexactinellida sp. 4',
                      'hexactinellida sp. 3', 
                      'asteroidea sp. 2',
                      'nanomia 2',
                      'oneirophanta mutabilis complex',
                      'holothuroidea sp. 4',
                      'mertensiidae sp. b',
                      'pennatulacea sp. 1',
                      'mycale sp. 1',
                      'protoptilum sp. 1',
                      'macellicephala sp. a',
                      'elpidia sp. a',
                      'lyssacinosida sp. 1',
                      'peniagone vitrea- sp. 1 complex',
                      'hexactinellida sp. 6',
                      'octacnemidae sp. 1',
                      'munida sp. a',
                      'doliolenetta',
                      'chondrocladia sp. a',
                      'peniagone sp. 2',
                      'neolithodes diomedae',
                      'acanthascinae sp. 3',
                      'asteroid',
                      'culeolus barryi',
                      'peniagone sp. 1',
                      'camptoplites sp. a',
                      'mycale sp. 2',
                      'fecampiid',
                      'umbellula sp. 1',
                      'forskalia edwardsi',
                      'holothuroidea sp. 6',
                      'acanthascinae sp. 1-4 complex',
                      'situla sp. 1',
                      'hexactinellida sp. 5',
                      'bathydorus sp. 1',
                      'culeolus sp. 2',
                      'prayidae sp. 1',
                      'hyalonema sp. 1',
                      'buccinidae sp. 1',
                      'zoantharia sp. 2',
                      'hexactinellida sp. 1',
                      'zoantharia sp. 1',
                      'acanthascinae sp. 2',
                      'pennatulacea sp. 3'
                     ]
nonorganism_concepts = [concept for concept in nonorganism_concepts if concept not in concepts_to_remove] 

# Add any terms that slipped through
nonorganism_concepts.extend(['iris', 'gastrozooid', 'cylinder', 'mars'])

In [14]:
## Remove nonorganism concepts from names

names = pd.Series(names)
names = names[~names.isin(nonorganism_concepts)]

In [42]:
%%capture cap --no-stderr --no-display

## Look up names in WoRMS and save matched name, name ID and taxon ID to dicts ----- TAKES ~ 15 MINUTES TO DO THE ENTIRE NAMES LIST

name_id_dic, name_dic, id_dic = WoRMS.run_get_worms_from_scientific_name(names, verbose_flag=True)

In [43]:
## Write failed names to log file

with open('VARS_2001_WoRMS_log_20200707.txt', 'w') as f:
    f.write(cap.stdout)

**Terms that seem like they should have matched but didn't from 2001, 2010, 2017, 1989:**
- hydromedusae (**Nancy identified the lowest taxononimc level as Leptolida. WoRMS does not accept this; the accepted name is Hydroidolina.**)
- phyllospadix-zostera (**Phyllospadix OR zostera**)
- gastrozooid (**This is an organism body part.**)
- clam (bivalvia)
- teuthoidea (**This is an old name. Teuthida**)
- medusae (**An MBARI term for a jelly that cannot be identified, even to class level. Phylum Cnidaria.**)
- larvacean house didn't match, but neither did larvacean (**Larvacea?**)
- neptunea-buccinum complex (**Can look complexes up on the MBARI Deep Sea Guide: http://m3.shore.mbari.org/dsg/view/concept/Neptunea-Buccinum%20Complex; family Buccinidae**)
- doliolinetta (**WoRMS will match Dolioletta and Doliolina. These are separate genuses. Doliolina is the same as Doliolinetta.**)
- tomopterid, tomopterid eggcase (**WoRMS will match Tomopteridae, is that correct?**)
- vitreosalpa gemini (**This is a special MBARI name for a species that has not been formally described yet. Family salpidae.**)
- sergestid (**WoRMS will match Sergestidae, is that correct?**)
- sebastomus complex (**Sebastes (also sebastomus) is a subgenus of Sebastes. Sebastes(Sebastomus) should match on WoRMS; aphia id = 991848**)
- graneledoninae (**This is an instance where MBARI may be more up to date than WoRMS. Use family Octopodidae.**)
- pyrosomida (**This is an old class name. Should be Pyrosomatida.**)
- funiculina-halipteris complex (**Family Funiculinidae**)
- torquaratoridaeb (**This is a genus in the family Torquaratoridae http://dsg.mbari.org/dsg/images/concept/Enteropneusta. There is also a torquaratoridaeA. Family Torquaratoridae**).
- nectophore (**This is an organism body part.**)
- lrj complex (**Little Red Jellies complex. Family Rhopalonematidae.**)
- brachiopod basket doesn't match, meaning that brachiopod won't
- lophophore 
- amphipod tube mat reduces to amphipod, but doesn't match

Also **note** that some terms that did match weren't necessarily species:
- cylinder (**This should not be matched; it refers to a piece of equipment.**)
- phyllospadix settlement rake matched to phyllospadix
- mars node matched to mars
- bathochordaeus sinker matched to bathochordaeus (**is this appropriate? Does this indicate just the mucus house?**)
- coryphaenoides acrolepis-filifer complex matched to coryphaenoides (**This is fine. Coryphaenoides is a genus of fish, and the species cannot easily be distinguished by eye. Just match the genus.**)
- coryphaenoides armatus-leptolepis-yaquinae complex matched to coryphaenoides
- coryphaenoides armatus-yaquinae complex matched to coryphaenoides
- medusa carcass matched to medusa
- phyllospadix-zostera detritus is reduced to phyllospadix-zostera and fails
- sergestid molt matched to sergestid
- sebastes aleutianus-melanostictus complex matched to sebastes (**This is fine.**)
- sebastes crameri-melanostomus complex matched to sebastes
- polychaeta tube matched to polychaeta
- opisthoteuthis cf. californiana matched to opisthoteuthis (**This is fine. cf. stands for the Latin conferre, meaning that the organism observed had features consistent with opistoteuthis californiana, but the species ID is uncertain.**)
- pyrosoma detritus matched to pyrosoma
- oneirophanta mutabilis complex matched to oneirophanta
- farrea truncata complex matched to farrea (**This is fine.**)

#### If desired, save dictionaries as json

In [44]:
## Save dictionaries

with open('VARS_2001_name_id_dict.json', 'w') as fp:
    json.dump(name_id_dic, fp, sort_keys=True, indent=4)
    
with open('VARS_2001_name_name_dict.json', 'w') as fp:
    json.dump(name_dic, fp, sort_keys=True, indent=4)
    
with open('VARS_2001_name_taxid_dict.json', 'w') as fp:
    json.dump(id_dic, fp, sort_keys=True, indent=4)

#### If desired, read dictionaries rather than querying WoRMS

In [15]:
## Load dictionaries

with open('VARS_2001_name_id_dict.json') as f:
  name_id_dic = json.load(f)

with open('VARS_2001_name_name_dict.json') as f:
  name_dic = json.load(f)

with open('VARS_2001_name_taxid_dict.json') as f:
  id_dic = json.load(f)

#### Handle organisms that should have matched on WoRMS but didn't

In [16]:
## Create a dictionary mapping biological names that didn't match on WoRMS to names that should match

VARS_to_WoRMS_dict = {'hydromedusae':'hydroidolina',
                      'teuthoidea':'teuthida',
                      'lrj complex':'rhopalonematidae',
                      'medusae':'cnidaria',
                      'neptunea-buccinum complex':'buccinidae',
                      'doliolinetta':'doliolina',
                      'doliolenetta':'doliolina',
                      'vitreosalpa gemini':'salpidae',
                      'vitreosalpa':'salpidae',
                      'sebastomus complex':'sebastes (sebastomus)',
                      'graneledoninae':'octopodidae',
                      'funiculina-halipteris complex':'funiculinidae',
                      'torquaratoridaea sp. 1':'torquaratoridae',
                      'torquaratoridaeb sp. 1':'torquaratoridae',
                      'torquaratoridaeb sp. 2':'torquaratoridae',
                      'funiculina-halipteris complex':'funiculinidae'
                     }

In [17]:
## Run these additional terms through WoRMS

revised_concepts = ['hydroidolina', 'teuthida', 'rhopalonematidae', 'cnidaria', 'buccinidae', 'doliolina', 'salpidae', 'sebastes (sebastomus)', 'octopodidae', 'funiculinidae',
                   'torquaratoridae', 'funiculinidae']
revised_name_id_dic, revised_name_dic, revised_id_dic = WoRMS.run_get_worms_from_scientific_name(revised_concepts, verbose_flag=True)

In [18]:
## Add values for revised names to original WoRMS output

name_id_dic.update(revised_name_id_dic)
name_dic.update(revised_name_dic)
id_dic.update(revised_id_dic)

In [19]:
## Create columns from WoRMS data

# Replace names that don't have a WoRMS match in scientificName with revised names
converted['scientificName'].replace(VARS_to_WoRMS_dict, inplace=True)

# Create scientificNameID column with the same content as scientificName - strip to ensure no whitespace, lowercase
converted['scientificNameID'] = converted['scientificName'].str.strip().str.lower()

# Use dictionary to replace scientific names with name IDs
converted.replace({'scientificNameID':name_id_dic}, inplace=True)

# Repeat to create taxonID
converted['taxonID'] = converted['scientificName'].str.strip().str.lower()
converted.replace({'taxonID':id_dic}, inplace=True)

converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode,occurrenceID,scientificName,occurrenceRemarks,identifiedBy,...,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters,scientificNameID,taxonID
0,2001-05-14T23:50:03Z,Tiburon_0324,ROV,David Clague,VARS,MBARI,53CF02AF-535F-41AE-B31E-4BCAB4F39A56,manipulator,manipulator,vars,...,-159.504933,2.07,320.799988,34.594002,2.407,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0324-09,300,manipulator,manipulator
1,2001-05-14T23:50:03Z,Tiburon_0324,ROV,David Clague,VARS,MBARI,E9881AC2-AD21-4420-B2DD-3C4A73645E92,holothuroidea,Holothuroidea,vars,...,-159.504933,2.07,320.799988,34.594002,2.407,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0324-09,300,urn:lsid:marinespecies.org:taxname:123083,123083
2,2001-07-05T21:16:45Z,Ventana_2016,ROV,Rob Sherlock,VARS,MBARI,5F374B81-172F-4888-8C2B-1489E9C8A366,forskalia,Forskalia,schlin,...,-122.059237,2.55,329.500000,34.019001,9.307,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2016-05,300,urn:lsid:marinespecies.org:taxname:135396,135396
5,2001-10-03T17:15:41Z,Ventana_2076,ROV,Charlie Paull,VARS,MBARI,C283DC75-BB9A-4E98-9E72-BC8EABC6EED9,rock,rock,svonthun,...,-121.984398,0.37,255.600006,34.513000,4.420,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2076-02,300,rock,rock
6,2001-10-03T17:15:41Z,Ventana_2076,ROV,Charlie Paull,VARS,MBARI,EBE21573-7B5D-422B-87ED-83CB16F1D611,ledge,ledge,svonthun,...,-121.984398,0.37,255.600006,34.513000,4.420,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2076-02,300,ledge,ledge


In [20]:
## Remove rows that didn't have a WoRMS match

converted = converted[converted['scientificName'].str.strip().str.lower() != converted['scientificNameID']]
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode,occurrenceID,scientificName,occurrenceRemarks,identifiedBy,...,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters,scientificNameID,taxonID
1,2001-05-14T23:50:03Z,Tiburon_0324,ROV,David Clague,VARS,MBARI,E9881AC2-AD21-4420-B2DD-3C4A73645E92,holothuroidea,Holothuroidea,vars,...,-159.504933,2.07,320.799988,34.594002,2.407,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0324-09,300,urn:lsid:marinespecies.org:taxname:123083,123083
2,2001-07-05T21:16:45Z,Ventana_2016,ROV,Rob Sherlock,VARS,MBARI,5F374B81-172F-4888-8C2B-1489E9C8A366,forskalia,Forskalia,schlin,...,-122.059237,2.55,329.500000,34.019001,9.307,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2016-05,300,urn:lsid:marinespecies.org:taxname:135396,135396
7,2001-07-19T20:20:27Z,Tiburon_0336,ROV,Bruce Robison,VARS,MBARI,1189576D-01D8-44BC-869D-5B2B5270AA18,hydroidolina,Hydromedusae,schlin,...,-122.528463,1.37,279.100006,34.584000,2.254,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0336-06,300,urn:lsid:marinespecies.org:taxname:19494,19494
9,2001-03-21T18:40:01Z,Tiburon_0266,ROV,Bruce Robison,VARS,MBARI,C8428DAB-39B5-46BC-A87F-C50A18E351BD,lobata,Lobata,schlin,...,-143.506874,5.27,254.699997,34.034000,11.690,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0266-01,300,urn:lsid:marinespecies.org:taxname:603346,603346
15,2001-07-19T17:32:11Z,Tiburon_0336,ROV,Bruce Robison,VARS,MBARI,1EB91809-BABF-4282-9AB9-802A5B4CFC2A,apolemia,Apolemia,svonthun,...,-122.529437,0.42,32.000000,34.469002,3.841,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0336-04,300,urn:lsid:marinespecies.org:taxname:135393,135393


In [21]:
## Replace scientificName with matched scientific names from WoRMS

converted['scientificName'] = converted['scientificName'].str.strip().str.lower()
converted['scientificName'].replace(name_dic, inplace=True)
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode,occurrenceID,scientificName,occurrenceRemarks,identifiedBy,...,decimalLongitude,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters,scientificNameID,taxonID
1,2001-05-14T23:50:03Z,Tiburon_0324,ROV,David Clague,VARS,MBARI,E9881AC2-AD21-4420-B2DD-3C4A73645E92,Holothuroidea,Holothuroidea,vars,...,-159.504933,2.07,320.799988,34.594002,2.407,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0324-09,300,urn:lsid:marinespecies.org:taxname:123083,123083
2,2001-07-05T21:16:45Z,Ventana_2016,ROV,Rob Sherlock,VARS,MBARI,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,Forskalia,schlin,...,-122.059237,2.55,329.500000,34.019001,9.307,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2016-05,300,urn:lsid:marinespecies.org:taxname:135396,135396
7,2001-07-19T20:20:27Z,Tiburon_0336,ROV,Bruce Robison,VARS,MBARI,1189576D-01D8-44BC-869D-5B2B5270AA18,Hydroidolina,Hydromedusae,schlin,...,-122.528463,1.37,279.100006,34.584000,2.254,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0336-06,300,urn:lsid:marinespecies.org:taxname:19494,19494
9,2001-03-21T18:40:01Z,Tiburon_0266,ROV,Bruce Robison,VARS,MBARI,C8428DAB-39B5-46BC-A87F-C50A18E351BD,Lobata,Lobata,schlin,...,-143.506874,5.27,254.699997,34.034000,11.690,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0266-01,300,urn:lsid:marinespecies.org:taxname:603346,603346
15,2001-07-19T17:32:11Z,Tiburon_0336,ROV,Bruce Robison,VARS,MBARI,1EB91809-BABF-4282-9AB9-802A5B4CFC2A,Apolemia,Apolemia,svonthun,...,-122.529437,0.42,32.000000,34.469002,3.841,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0336-04,300,urn:lsid:marinespecies.org:taxname:135393,135393


In [22]:
## Create additional needed columns

converted['nameAccordingTo'] = 'WoRMS'
converted['occurrenceStatus'] = 'present'
converted['basisOfRecord'] = 'MachineObservation'

converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode,occurrenceID,scientificName,occurrenceRemarks,identifiedBy,...,salinity,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters,scientificNameID,taxonID,nameAccordingTo,occurrenceStatus,basisOfRecord
1,2001-05-14T23:50:03Z,Tiburon_0324,ROV,David Clague,VARS,MBARI,E9881AC2-AD21-4420-B2DD-3C4A73645E92,Holothuroidea,Holothuroidea,vars,...,34.594002,2.407,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0324-09,300,urn:lsid:marinespecies.org:taxname:123083,123083,WoRMS,present,MachineObservation
2,2001-07-05T21:16:45Z,Ventana_2016,ROV,Rob Sherlock,VARS,MBARI,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,Forskalia,schlin,...,34.019001,9.307,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2016-05,300,urn:lsid:marinespecies.org:taxname:135396,135396,WoRMS,present,MachineObservation
7,2001-07-19T20:20:27Z,Tiburon_0336,ROV,Bruce Robison,VARS,MBARI,1189576D-01D8-44BC-869D-5B2B5270AA18,Hydroidolina,Hydromedusae,schlin,...,34.584000,2.254,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0336-06,300,urn:lsid:marinespecies.org:taxname:19494,19494,WoRMS,present,MachineObservation
9,2001-03-21T18:40:01Z,Tiburon_0266,ROV,Bruce Robison,VARS,MBARI,C8428DAB-39B5-46BC-A87F-C50A18E351BD,Lobata,Lobata,schlin,...,34.034000,11.690,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0266-01,300,urn:lsid:marinespecies.org:taxname:603346,603346,WoRMS,present,MachineObservation
15,2001-07-19T17:32:11Z,Tiburon_0336,ROV,Bruce Robison,VARS,MBARI,1EB91809-BABF-4282-9AB9-802A5B4CFC2A,Apolemia,Apolemia,svonthun,...,34.469002,3.841,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0336-04,300,urn:lsid:marinespecies.org:taxname:135393,135393,WoRMS,present,MachineObservation


In [23]:
## Assemble associatedMedia

associatedMedia = []

for occ_id in converted['occurrenceID'].unique():
    
    # Select data associated with that occurrenceID:
    selected = converted[converted['occurrenceID'] == occ_id]
    
    # Retrieve unique image and video files
    image_files = selected['image_url'].drop_duplicates()
    video_files = selected['video_uri'].drop_duplicates()
    
    # Remove any NaN values
    image_files = image_files.dropna()
    video_files = video_files.dropna()
    
    # Join image and video files
    media = pd.concat([image_files, video_files])
    
    # Create a string with all the urls
    url_str = ''
    for url in media: url_str = url_str + url + ' | '
    url_str = url_str[0:-3]
    
    # Add to associatedMedia
    associatedMedia.append(url_str)

**Note:** That took a little under 15 minutes to run. I don't know if there's a better/easier way to do this?

In [24]:
## Add to df

# First, need to remove rows with duplicate occurrenceIDs
converted = converted.drop_duplicates(subset='occurrenceID', keep="first")

# Add associatedMedia
converted['associatedMedia'] = associatedMedia
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode,occurrenceID,scientificName,occurrenceRemarks,identifiedBy,...,temperatureInCelsius,image_url,video_uri,coordinateUncertaintyInMeters,scientificNameID,taxonID,nameAccordingTo,occurrenceStatus,basisOfRecord,associatedMedia
1,2001-05-14T23:50:03Z,Tiburon_0324,ROV,David Clague,VARS,MBARI,E9881AC2-AD21-4420-B2DD-3C4A73645E92,Holothuroidea,Holothuroidea,vars,...,2.407,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0324-09,300,urn:lsid:marinespecies.org:taxname:123083,123083,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...
2,2001-07-05T21:16:45Z,Ventana_2016,ROV,Rob Sherlock,VARS,MBARI,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,Forskalia,schlin,...,9.307,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...,urn:tid:mbari.org:V2016-05,300,urn:lsid:marinespecies.org:taxname:135396,135396,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...
7,2001-07-19T20:20:27Z,Tiburon_0336,ROV,Bruce Robison,VARS,MBARI,1189576D-01D8-44BC-869D-5B2B5270AA18,Hydroidolina,Hydromedusae,schlin,...,2.254,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0336-06,300,urn:lsid:marinespecies.org:taxname:19494,19494,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...
9,2001-03-21T18:40:01Z,Tiburon_0266,ROV,Bruce Robison,VARS,MBARI,C8428DAB-39B5-46BC-A87F-C50A18E351BD,Lobata,Lobata,schlin,...,11.690,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0266-01,300,urn:lsid:marinespecies.org:taxname:603346,603346,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...
15,2001-07-19T17:32:11Z,Tiburon_0336,ROV,Bruce Robison,VARS,MBARI,1EB91809-BABF-4282-9AB9-802A5B4CFC2A,Apolemia,Apolemia,svonthun,...,3.841,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...,urn:tid:mbari.org:T0336-04,300,urn:lsid:marinespecies.org:taxname:135393,135393,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...


In [25]:
## Save columns for MeasurementOrFact file

mof = converted[['occurrenceID', 'dissolvedOxygenInMLPerL', 'pressureInPsi', 'salinity', 'temperatureInCelsius']]

In [26]:
## Drop extra columns

converted = converted.drop(['image_url', 'video_uri', 'dissolvedOxygenInMLPerL', 'pressureInPsi', 'salinity', 'temperatureInCelsius'], axis=1)
converted.head()

,eventDate,eventID,samplingProtocol,recordedBy,datasetID,institutionCode,occurrenceID,scientificName,occurrenceRemarks,identifiedBy,...,verbatimDepth,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,scientificNameID,taxonID,nameAccordingTo,occurrenceStatus,basisOfRecord,associatedMedia
1,2001-05-14T23:50:03Z,Tiburon_0324,ROV,David Clague,VARS,MBARI,E9881AC2-AD21-4420-B2DD-3C4A73645E92,Holothuroidea,Holothuroidea,vars,...,1948.6,21.742367,-159.504933,300,urn:lsid:marinespecies.org:taxname:123083,123083,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...
2,2001-07-05T21:16:45Z,Ventana_2016,ROV,Rob Sherlock,VARS,MBARI,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,Forskalia,schlin,...,98.4,36.702446,-122.059237,300,urn:lsid:marinespecies.org:taxname:135396,135396,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...
7,2001-07-19T20:20:27Z,Tiburon_0336,ROV,Bruce Robison,VARS,MBARI,1189576D-01D8-44BC-869D-5B2B5270AA18,Hydroidolina,Hydromedusae,schlin,...,1825.4,36.569788,-122.528463,300,urn:lsid:marinespecies.org:taxname:19494,19494,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...
9,2001-03-21T18:40:01Z,Tiburon_0266,ROV,Bruce Robison,VARS,MBARI,C8428DAB-39B5-46BC-A87F-C50A18E351BD,Lobata,Lobata,schlin,...,264.3,28.181265,-143.506874,300,urn:lsid:marinespecies.org:taxname:603346,603346,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...
15,2001-07-19T17:32:11Z,Tiburon_0336,ROV,Bruce Robison,VARS,MBARI,1EB91809-BABF-4282-9AB9-802A5B4CFC2A,Apolemia,Apolemia,svonthun,...,1063.7,36.569538,-122.529437,300,urn:lsid:marinespecies.org:taxname:135393,135393,WoRMS,present,MachineObservation,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...


In [27]:
## Reorder columns

converted = converted[['eventID', 'eventDate', 'samplingProtocol', 'recordedBy', 'datasetID', 'institutionCode', 'occurrenceID', 'scientificName', 'scientificNameID', 'taxonID', 
                       'nameAccordingTo', 'occurrenceStatus', 'basisOfRecord', 'identifiedBy', 'occurrenceRemarks', 'decimalLatitude', 'decimalLongitude', 'coordinateUncertaintyInMeters',
                       'minimumDepthInMeters', 'maximumDepthInMeters', 'verbatimDepth', 'associatedMedia']]
converted.head()

,eventID,eventDate,samplingProtocol,recordedBy,datasetID,institutionCode,occurrenceID,scientificName,scientificNameID,taxonID,...,basisOfRecord,identifiedBy,occurrenceRemarks,decimalLatitude,decimalLongitude,coordinateUncertaintyInMeters,minimumDepthInMeters,maximumDepthInMeters,verbatimDepth,associatedMedia
1,Tiburon_0324,2001-05-14T23:50:03Z,ROV,David Clague,VARS,MBARI,E9881AC2-AD21-4420-B2DD-3C4A73645E92,Holothuroidea,urn:lsid:marinespecies.org:taxname:123083,123083,...,MachineObservation,vars,Holothuroidea,21.742367,-159.504933,300,1946.4,1950.8,1948.6,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...
2,Ventana_2016,2001-07-05T21:16:45Z,ROV,Rob Sherlock,VARS,MBARI,5F374B81-172F-4888-8C2B-1489E9C8A366,Forskalia,urn:lsid:marinespecies.org:taxname:135396,135396,...,MachineObservation,schlin,Forskalia,36.702446,-122.059237,300,96.2,100.6,98.4,http://search.mbari.org/ARCHIVE/frameGrabs/Ven...
7,Tiburon_0336,2001-07-19T20:20:27Z,ROV,Bruce Robison,VARS,MBARI,1189576D-01D8-44BC-869D-5B2B5270AA18,Hydroidolina,urn:lsid:marinespecies.org:taxname:19494,19494,...,MachineObservation,schlin,Hydromedusae,36.569788,-122.528463,300,1823.2,1827.6,1825.4,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...
9,Tiburon_0266,2001-03-21T18:40:01Z,ROV,Bruce Robison,VARS,MBARI,C8428DAB-39B5-46BC-A87F-C50A18E351BD,Lobata,urn:lsid:marinespecies.org:taxname:603346,603346,...,MachineObservation,schlin,Lobata,28.181265,-143.506874,300,262.1,266.5,264.3,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...
15,Tiburon_0336,2001-07-19T17:32:11Z,ROV,Bruce Robison,VARS,MBARI,1EB91809-BABF-4282-9AB9-802A5B4CFC2A,Apolemia,urn:lsid:marinespecies.org:taxname:135393,135393,...,MachineObservation,svonthun,Apolemia,36.569538,-122.529437,300,1061.5,1065.9,1063.7,http://search.mbari.org/ARCHIVE/frameGrabs/Tib...


In [28]:
## Save occurrence file

converted.to_csv('VARS_2001_converted_20200720.csv', index=False, na_rep='NaN')

**Note:** This pipeline takes 20-25 minutes to run on ~200,000 records. Primarily, this time comes from 1) WoRMS look-up and 2) assembly of associatedMedia.

### Build MeasurementOrFact file

In [29]:
## Add columns by occurrenceID

mof.head()

,occurrenceID,dissolvedOxygenInMLPerL,pressureInPsi,salinity,temperatureInCelsius
1,E9881AC2-AD21-4420-B2DD-3C4A73645E92,2.07,320.799988,34.594002,2.407
2,5F374B81-172F-4888-8C2B-1489E9C8A366,2.55,329.500000,34.019001,9.307
7,1189576D-01D8-44BC-869D-5B2B5270AA18,1.37,279.100006,34.584000,2.254
9,C8428DAB-39B5-46BC-A87F-C50A18E351BD,5.27,254.699997,34.034000,11.690
15,1EB91809-BABF-4282-9AB9-802A5B4CFC2A,0.42,32.000000,34.469002,3.841


In [30]:
## Convert to long format

mof_long = pd.melt(mof, id_vars='occurrenceID', var_name='measurementType', value_name='measurementValue')
mof_long.head()

,occurrenceID,measurementType,measurementValue
0,E9881AC2-AD21-4420-B2DD-3C4A73645E92,dissolvedOxygenInMLPerL,2.07
1,5F374B81-172F-4888-8C2B-1489E9C8A366,dissolvedOxygenInMLPerL,2.55
2,1189576D-01D8-44BC-869D-5B2B5270AA18,dissolvedOxygenInMLPerL,1.37
3,C8428DAB-39B5-46BC-A87F-C50A18E351BD,dissolvedOxygenInMLPerL,5.27
4,1EB91809-BABF-4282-9AB9-802A5B4CFC2A,dissolvedOxygenInMLPerL,0.42


There are ~200 rows where salinity=0. That doesn't make sense, and I'm assuming salinity is unavailable for these records. Changing them to NaN.

In [31]:
## Change salinity = 0 to NaN

mof_long.loc[(mof_long['measurementType'] == 'salinity') & (mof_long['measurementValue'] == 0), 'measurementValue'] = np.nan

In [32]:
## Round

mof_long['measurementValue'] = round(mof_long['measurementValue'], 2)

In [33]:
## Change measurementType names

mof_long['measurementType'].replace({
    'dissolvedOxygenInMLPerL':'dissolvedOxygen',
    'pressureInPsi':'pressure',
    'temperatureInCelsius':'temperature'
}, inplace = True)

In [34]:
## Add measurementUnit

mof_long['measurementUnit'] = 'mL per L seawater'
mof_long.loc[mof_long['measurementType'] == 'pressure', 'measurementUnit'] = 'psi'
mof_long.loc[mof_long['measurementType'] == 'salinity', 'measurementUnit'] = 'psu'
mof_long.loc[mof_long['measurementType'] == 'temperature', 'measurementUnit'] = 'celsius'

mof_long.head()

,occurrenceID,measurementType,measurementValue,measurementUnit
0,E9881AC2-AD21-4420-B2DD-3C4A73645E92,dissolvedOxygen,2.07,mL per L seawater
1,5F374B81-172F-4888-8C2B-1489E9C8A366,dissolvedOxygen,2.55,mL per L seawater
2,1189576D-01D8-44BC-869D-5B2B5270AA18,dissolvedOxygen,1.37,mL per L seawater
3,C8428DAB-39B5-46BC-A87F-C50A18E351BD,dissolvedOxygen,5.27,mL per L seawater
4,1EB91809-BABF-4282-9AB9-802A5B4CFC2A,dissolvedOxygen,0.42,mL per L seawater


In [35]:
## Save

mof_long.to_csv('VARS_2001_MoF_20200720.csv', index=False, na_rep='NaN')

### Remaining issues

1. coordinateUncertaintyInMeters **Talked with Dave Caress and Jenny Paduan about this. It is an extremely complex issue. The short story is that it's probably possible to calculate the *best* possible error from first principles, and then talk to Linda Kuntz (recently retired) to figure out which dives had known problems and the best possible error shouldn't be applied. When pressed for an estimate of *worst* possible error, it sounds like it varies with depth and tether length, but could be up to 300 m. After consulting with Patrick, it seems like the best approach is to give the upper limit of 300 m for now.**
2. How accurate are depths? Probably not to 6 decimal places. **Talked to Dave Caress and Jenny Paduan about this. The accuracy of pressure sensors is some percentage of their range; for the sensors on MBARI's ROVs and AUVs, their error is in the range of 1-10 cm. Bear in mind, though, that the pressure sensor is on top of the ROV (approximately 2 meters off the bottom), and that tide has not been accounted for. Tidal range in Monterey Bay is a little under 8 ft, or about 2.5 m. In the past, Jenny has rounded depth**
3. Check through WoRMS log for names that should have matched **Got a detailed response to my questions from Nancy Jacobsen-Stout. Have incorporated these changes.**
4. Faster way to assemble associatedMedia?

**Note:** The WoRMS lookup could probably be significantly accelerated by compiling a list of rejected terms from the WoRMS logs I've already created, and removing matching records before querying WoRMS. **Completed - only sped things up a little.**